In [8]:
from pythreejs import *
import numpy as np
from IPython.display import display
from IPython.html.widgets import *

In [9]:
nx,ny=(20,20)
xmax=1
x = np.linspace(-xmax,xmax,nx)
y = np.linspace(-xmax,xmax,ny)
xx, yy = np.meshgrid(x,y)
z = xx**2-yy**2
g = SurfaceGeometry(z=list(z[::-1].flat), 
                          width=2*xmax,
                          height=2*xmax,
                          width_segments=nx-1,
                          height_segments=ny-1)
surf = Mesh(geometry=g, material=LambertMaterial(color='white'))
children = [surf, AmbientLight(color=0x777777)]
children.append(SurfaceGrid(geometry=g, material=LineBasicMaterial(color='black')))

scene = Scene(children=children)
c = PerspectiveCamera(position=[0,5,5], up=[0,0,1], children=[DirectionalLight(color='white', 
                                                                             position=[3,5,1], 
                                                                             intensity=0.5)])
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c)])
display(renderer)

In [10]:
g = SphereGeometry(radius=1)
surf = Mesh(geometry=g, material=LambertMaterial(color='red'), position=[2,1,0])
children = [surf, AmbientLight(color=0x777777), make_text('Hello World!', height=.6)]
scene = Scene(children=children)
c = PerspectiveCamera(position=[0,5,5], up=[0,0,1], children=[DirectionalLight(color='white', 
                                                                             position=[3,5,1], 
                                                                             intensity=0.5)])
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c)])
display(renderer)